## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/7/19,CP BL,7.47,NaN,54.0,0.90,8.30,Specialized
1,10/20/16,Citibike to Work,1.15,NaN,9.0,0.15,7.67,Citibike
2,8/28/18,CP BL,6.78,NaN,43.0,0.72,9.46,Specialized
3,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized
4,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
...,...,...,...,...,...,...,...,...
348,6/21/16,Citibike from Work,0.78,NaN,7.0,0.12,6.69,Citibike
349,9/7/20,Wards Island x 2,17.75,2.0,10.0,2.17,8.19,Specialized
350,6/22/19,Down ES UP WS,16.23,2.0,28.0,2.47,6.58,Specialized
351,11/4/16,Citibike to Work,1.12,NaN,10.0,0.17,6.72,Citibike


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
1,10/20/16,Citibike to Work,1.15,0.0,9.0,0.15,7.67,Citibike
2,8/28/18,CP BL,6.78,0.0,43.0,0.72,9.46,Specialized
3,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized
4,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
...,...,...,...,...,...,...,...,...
348,6/21/16,Citibike from Work,0.78,0.0,7.0,0.12,6.69,Citibike
349,9/7/20,Wards Island x 2,17.75,2.0,10.0,2.17,8.19,Specialized
350,6/22/19,Down ES UP WS,16.23,2.0,28.0,2.47,6.58,Specialized
351,11/4/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
1,1,10/20/16,Citibike to Work,1.15,0.0,9.0,0.15,7.67,Citibike
2,2,8/28/18,CP BL,6.78,0.0,43.0,0.72,9.46,Specialized
3,3,8/27/19,CP BL x 2,13.66,1.0,49.0,1.82,7.52,Specialized
4,4,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
...,...,...,...,...,...,...,...,...,...
348,348,6/21/16,Citibike from Work,0.78,0.0,7.0,0.12,6.69,Citibike
349,349,9/7/20,Wards Island x 2,17.75,2.0,10.0,2.17,8.19,Specialized
350,350,6/22/19,Down ES UP WS,16.23,2.0,28.0,2.47,6.58,Specialized
351,351,11/4/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,7.47,0.90,8.30,Specialized
1,1.15,0.15,7.67,Citibike
2,6.78,0.72,9.46,Specialized
3,13.66,1.82,7.52,Specialized
4,13.35,1.50,8.90,Specialized
...,...,...,...,...
348,0.78,0.12,6.69,Citibike
349,17.75,2.17,8.19,Specialized
350,16.23,2.47,6.58,Specialized
351,1.12,0.17,6.72,Citibike


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
#new2_df = all_df[['Miles', 'Duration', 'Speed', 'Type']]
#new2_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(264, 3)

In [13]:
# Checking testing data shape
X_test.shape

(89, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9662921348314607


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
233,Specialized
290,Citibike
122,Specialized
158,Specialized
95,Specialized
...,...
313,Citibike
216,Citibike
49,Specialized
145,Citibike


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,233,Specialized
1,290,Citibike
2,122,Specialized
3,158,Specialized
4,95,Specialized
...,...,...
84,313,Citibike
85,216,Citibike
86,49,Specialized
87,145,Citibike


### Combining the test and prediction dataframes for comparison

In [21]:
# Combining the new test and predition dataframes horizontally
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,233,Specialized,Specialized
1,290,Citibike,Citibike
2,122,Specialized,Specialized
3,158,Specialized,Specialized
4,95,Specialized,Specialized
...,...,...,...
84,313,Citibike,Citibike
85,216,Citibike,Citibike
86,49,Specialized,Specialized
87,145,Citibike,Citibike


In [22]:
# Exporting Combined Test and Prediction Dataframe for examination
test_pred_file = "test_pred.csv"
test_pred_df.to_csv(os.path.join(data_path, test_pred_file))

In [23]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
13,151,Specialized,Citibike
48,295,Specialized,Citibike
79,132,Citibike,Specialized


In [24]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,151,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80,Specialized
1,295,Specialized,Citibike,6/28/20,Downtown Trio,4.10,0.0,34.0,0.57,7.24,Specialized
2,132,Citibike,Specialized,6/17/18,Work to Village,4.69,0.0,40.0,0.67,7.04,Citibike


In [25]:
# Cleaning up err_details_df
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,151,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80
1,295,Specialized,Citibike,6/28/20,Downtown Trio,4.10,0.0,34.0,0.57,7.24
2,132,Citibike,Specialized,6/17/18,Work to Village,4.69,0.0,40.0,0.67,7.04
